In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Literal
from dotenv import load_dotenv
import os

c:\ProgramData\anaconda3\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
load_dotenv('../../credentials.env')

True

In [4]:
def system_prompt_builder(input_text, available_attachments):
    system_prompt = f"""
    You are a medical input classifier. Classify into ONE label.

    TEXT_VALID_ATTACHMENT_VALID
    - Text shows medical intent (symptoms, diagnosis, treatment, test results, health questions)
    - Attachments absent OR have medical-relevant names

    TEXT_VALID_ATTACHMENT_NOT_VALID
    - Text shows medical intent
    - Attachments present with clearly non-medical names (vacation.jpg, recipe.pdf)

    TEXT_NOT_VALID_ATTACHMENT_VALID
    - Text is non-medical (greetings, jokes, coding, sports, general chat)
    - Attachments have medical names (xray, report, scan, lab, test)
    - User likely uploaded wrong files or is confused

    TEXT_NOT_VALID_ATTACHMENT_NOT_VALID
    - Text is non-medical
    - Attachments absent OR clearly non-medical

    Rules:
    - "xray", "report", "scan", "lab", "test" in filenames = medical attachment
    - When text is invalid but attachments are medical, still classify as TEXT_NOT_VALID_ATTACHMENT_VALID

    user input: {input_text}
    available attachments: {available_attachments}
    """
    return system_prompt

In [5]:
class MedicalInputCheck(TypedDict):
    input_classification: Literal["TEXT_VALID_ATTACHMENT_VALID", "TEXT_VALID_ATTACHMENT_NOT_VALID", "TEXT_NOT_VALID_ATTACHMENT_VALID", "TEXT_NOT_VALID_ATTACHMENT_NOT_VALID"]

In [6]:
structured_llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    
    ).with_structured_output(MedicalInputCheck)

In [7]:
from langchain_openai import ChatOpenAI

qwen_llm = ChatOpenAI(
    base_url="http://127.0.0.1:1234/v1",
    model="qwen/qwen3-4b-2507",
    api_key="f"
).with_structured_output(MedicalInputCheck)

In [8]:
input_text = "am i okay?"
available_attachments = ["xray.png", "my blood image.pdf"]

In [ ]:
available_attachments = available_attachments if available_attachments else "The user did not provide any attachments."
print(available_attachments)
result = structured_llm.invoke([
    ("system", system_prompt_builder(input_text, available_attachments)),
    ("human", input_text)
])

print(result)

['xray.png', 'my blood image.pdf']
{'input_classification': 'TEXT_VALID_ATTACHMENT_VALID'}


In [12]:
print(type(result['input_classification']))

<class 'str'>


In [3]:
system_prompt_text_only = f"""
You are a medical text validator. Classify the input into ONE label.

TEXT_VALID
- Text shows medical intent (symptoms, diagnosis, treatment, test results, health questions)

TEXT_NOT_VALID
- Text is non-medical (greetings, jokes, coding, sports, general chat)

Rules:
- Don't be too strict, classify it as not valid only if the input is very clearly not a midical input
- Output exactly one label: TEXT_VALID or TEXT_NOT_VALID
"""

class TextOnlyInputValidator(TypedDict):
    input_classification: Literal["TEXT_VALID", "TEXT_NOT_VALID"]

structured_llm_text_only = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite-preview-09-2025",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    
    ).with_structured_output(TextOnlyInputValidator)

In [4]:
def validate_input_text_only(input_text: str):


    result = structured_llm_text_only.invoke([
        ("system", system_prompt_text_only),
        ("human", f"Input Text: {input_text}"),
        
    ])

    return result['input_classification']

In [13]:
results = validate_input_text_only("This is an extracted text from a pdf showing ALT Levels")

print(results)

TEXT_VALID
